In [1]:
from pandas import Series,DataFrame
import pandas as pd
import numpy as np

## pandas的数据结构介绍
`pandas`的两个主要数据结构：`Series`和`DataFrame`。

## Series
`Series`是一种类似于一维数组的对象，它由一组数据(各种`Numpy`数据类型)以及一组与之相关的数据标签(即索引)组成。仅由一组数据即可产生最简单的`Series`:

In [2]:
obj = Series([4, 7, -5, 3])
obj

0    4
1    7
2   -5
3    3
dtype: int64

`Series`的字符串表现形式为：索引在左边，值在右边。由于没有为数据指定索引，于是会自动创建一个0到`N-1`(N为数据的长度)的整数型索引。可以通过`Series`的`values`和`index`属性获取其数组表示形式和索引对象：

In [3]:
obj.values

array([ 4,  7, -5,  3], dtype=int64)

In [4]:
obj.index

RangeIndex(start=0, stop=4, step=1)

创建`Series`带有一个可以对各个数据点进行标记的索引：

In [5]:
obj2 = Series([4, 7, -5, 3], index=['d', 'b', 'a', 'c'])
obj2

d    4
b    7
a   -5
c    3
dtype: int64

In [6]:
obj2.index

Index(['d', 'b', 'a', 'c'], dtype='object')

与普通`NumPy`数组相比，可通过索引的方式选取`Series`中的单个或一组值：

In [7]:
obj2['a']

-5

In [8]:
obj2['d'] = 6

In [9]:
obj2[['c', 'a', 'd']]

c    3
a   -5
d    6
dtype: int64

`NumPy`数组运算都会保留索引和值之间的链接：

In [10]:
obj2

d    6
b    7
a   -5
c    3
dtype: int64

In [11]:
obj2[obj2 > 0]

d    6
b    7
c    3
dtype: int64

In [12]:
obj2 * 2

d    12
b    14
a   -10
c     6
dtype: int64

In [13]:
np.exp(obj2)  # e的x幂次方

d     403.428793
b    1096.633158
a       0.006738
c      20.085537
dtype: float64

还可将`Series`看成是一个定长的有序字典，因为它是索引值到数据值的一个映射。它可以用在许多原本需要字典参数的函数中：

In [14]:
'b' in obj2

True

In [15]:
'e' in obj2

False

若数据被存放在`Python`字典中，也可直接通过这个字典来创建`Series`:

In [16]:
sdata = {'Ohio':35000, 'Texas':71000, 'Oregon':16000, 'Utah':5000}
obj3 = Series(sdata)
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

若只传入一个字典，则结果`Series`中的索引就是原字典的键(有序排列)。

In [17]:
states = ['California', 'Ohio', 'Oregon', 'Texas']
obj4 = Series(sdata, index=states)
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

此例中，`sdata`中跟`states`索引相匹配的那3个值会被找出来并放到相应的位置上，但由于`“California”`所对应的`sdata`值找不到，所以其结果就为`NaN`(即“非数字”(`not a number`)，在`pandas`中，它用于表示缺失或`NA`值)。使用缺失(`missing`)或`NA`表示缺失数据。`pandas`的`isnull`和`notnull`函数可用于检测缺失数据：

In [18]:
pd.isnull(obj4)   # 是否缺失数据

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [19]:
pd.notnull(obj4)   # 是否没有缺失数据

California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool

In [20]:
obj4.isnull()

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

`Series`最重要的一个功能是:它在算数运算中会自动对齐不同索引的数据。

In [21]:
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

In [22]:
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

In [23]:
obj3 + obj4

California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

`Series`对象本身及其索引都有一个`name`属性，该属性跟`pandas`其它的关键功能关系非常密切：

In [24]:
obj4.name = 'population'   # series对象叫'population'
obj4.index.name = 'state'  # series对象的索引对象叫'state'
obj4

state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64

`Series`的索引可以通过赋值的方式就地修改：

In [25]:
obj.index = ['Bob', 'Steve', 'Jeff', 'Ryan']
obj

Bob      4
Steve    7
Jeff    -5
Ryan     3
dtype: int64

## DataFrame
`DataFrame`是表格型数据结构，列有序，每列可为不同的值类型。`DataFrame`既有行索引也有列索引,它可以被看做由`Series`组成的字典
（共用同一个索引）。`DataFrame`中面向行和面向列的操作基本上是平衡的。`DataFrame`中的数据是以一个或多个二维块存放的。  
注：虽然`DataFrame`是以二维结构保存数据的，但仍可以将其表示为更高维度的数据。

直接传入一个由等长列表或`NumPy`数组组成的字典，构建`DataFrame`:

In [26]:
data = {
    'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
    'year': [2000, 2001, 2002, 2001, 2002],
    'pop': [1.5, 1.7, 3.6, 2.4, 2.9]
}
frame = DataFrame(data)  # DataFrame会自动加上索引(如Series)，且全部列会被有序排列
frame

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9


若指定了列，则`DataFrame`的列就会按照指定顺序进行排列：

In [27]:
DataFrame(data, columns=['year', 'state', 'pop'])

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9


如`Series`, 若传入的列在数据中找不到，就会产生`NA`值：

In [28]:

frame2 = DataFrame(data, columns=['year', 'state', 'pop', 'debt'],
                  index=['one', 'two', 'three', 'four', 'five'])
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN


In [29]:
frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

通过类似字典标记的方式或属性的方式，可以将`DataFrame`的列获取为一个`Series`：

In [30]:
frame2['state']

one        Ohio
two        Ohio
three      Ohio
four     Nevada
five     Nevada
Name: state, dtype: object

In [31]:
frame2.year

one      2000
two      2001
three    2002
four     2001
five     2002
Name: year, dtype: int64

返回的`Series`拥有原`DataFrame`相同的索引，且其`name`属性也被相应的设置好了。行也可通过位置或名称的方式进行获取，如用索引字段`loc`：

In [32]:
frame2.loc['three']

year     2002
state    Ohio
pop       3.6
debt      NaN
Name: three, dtype: object

列可以通过赋值的方式修改，附上一个标量或一组值：

In [33]:
frame2['debt'] = 16.5
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,16.5
two,2001,Ohio,1.7,16.5
three,2002,Ohio,3.6,16.5
four,2001,Nevada,2.4,16.5
five,2002,Nevada,2.9,16.5


In [34]:
frame2['debt'] = np.arange(5.)
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,0.0
two,2001,Ohio,1.7,1.0
three,2002,Ohio,3.6,2.0
four,2001,Nevada,2.4,3.0
five,2002,Nevada,2.9,4.0


将列表或数组赋值给某列时，其长度必须跟`DataFrame`长度相匹配。若赋值的是一个`Series`，就会精确匹配`DataFrame`的索引，所有的空位都将被填上缺失值：

In [35]:
val = Series([-1.2, -1.5, -1.7], index=['two', 'four', 'five'])
frame2['debt'] = val
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,-1.2
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,-1.5
five,2002,Nevada,2.9,-1.7


为不存在的列赋值会创建出一个新列。关键字`del`用于删除列：

In [36]:
frame2['eastern'] = frame2.state == 'Ohio'
frame2

,year,state,pop,debt,eastern
one,2000,Ohio,1.5,NaN,True
two,2001,Ohio,1.7,-1.2,True
three,2002,Ohio,3.6,NaN,True
four,2001,Nevada,2.4,-1.5,False
five,2002,Nevada,2.9,-1.7,False


In [37]:
del frame2['eastern']
frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

注：通过索引方式返回的列只是相应数据的视图，并不是副本。因此，对返回的`Series`所做的任何就地修改全都会反映到源`DataFrame`上。通过
`Series`的`copy`方法即可显式的复制列。

另一种常见的数据形式是嵌套字典（字典的字典）：

In [38]:
pop = {'Nevada': {2001: 2.4, 2002: 2.9},
      'Ohio': {2000: 1.5, 2001: 1.7, 2002: 3.6}}

若将它传给`DataFrame`，他就会被解释为：外层字典的键作为列，内层键则作为行索引：

In [39]:
frame3 = DataFrame(pop)
frame3

,Nevada,Ohio
2001,2.4,1.7
2002,2.9,3.6
2000,NaN,1.5


也可对结果进行转置：

In [40]:
frame3.T

,2001,2002,2000
Nevada,2.4,2.9,NaN
Ohio,1.7,3.6,1.5


内层字典的键会被合并、排序以形成最终的索引。若显示指定了索引，则不会这样：

In [41]:
DataFrame(pop, index=[2001, 2002, 2003])

,Nevada,Ohio
2001,2.4,1.7
2002,2.9,3.6
2003,NaN,NaN


由`Series`组成的字典用法差不多：

In [42]:
pdata = {'Ohio': frame3['Ohio'][:-1],    # 从末尾-1但不包括
        'Nevada': frame3['Nevada'][:2]}
DataFrame(pdata)

,Ohio,Nevada
2001,1.7,2.4
2002,3.6,2.9


![](./可以输入给DataFrame构造器的数据.png)

若设置了`DataFrame`的`index`和`columns`的`name`属性，则这些信息也会被显示出来：

In [43]:
frame3.index.name = 'year'; frame3.columns.name = 'state'
frame3

state,Nevada,Ohio
year,,
2001,2.4,1.7
2002,2.9,3.6
2000,NaN,1.5


与`Series`一样，`values`属性也会以二维`ndarray`的形式返回`DataFrame`中的数据：

In [44]:
frame3.values

array([[2.4, 1.7],
       [2.9, 3.6],
       [nan, 1.5]])

若`DataFrame`各列的数据类型不同，则值数组的数据类型就会选用能兼容所有列的数据类型：

In [45]:
frame2.values

array([[2000, 'Ohio', 1.5, nan],
       [2001, 'Ohio', 1.7, -1.2],
       [2002, 'Ohio', 3.6, nan],
       [2001, 'Nevada', 2.4, -1.5],
       [2002, 'Nevada', 2.9, -1.7]], dtype=object)

## 索引对象
`pandas`的索引对象负责管理轴标签和其它元数据（如轴名称等）。构建`Series`或`DataFrame`时，所用到的任何数组或其他序列标签都会被转换成一个`Index`:

In [46]:
obj = Series(range(3), index=['a', 'b', 'c'])
index = obj.index
index

Index(['a', 'b', 'c'], dtype='object')

In [47]:
index[1:]

Index(['b', 'c'], dtype='object')

`Index`对象时不可修改的，因此用户不能对其进行修改，因其不可修改，故`Index`对象可在多个数据结构之间安全共享：

In [48]:
index = pd.Index(np.arange(3))
obj2 = Series([1.5, -2.5, 0], index=index)
obj2.index is index

True

`Index`甚至可被继承从而实现特别的轴索引功能。
![](./pandas中主要的index对象.png)

`Index`的功能也类似一个固定大小的集合：

In [49]:
frame3

state,Nevada,Ohio
year,,
2001,2.4,1.7
2002,2.9,3.6
2000,NaN,1.5


In [50]:
'Ohio' in frame3.columns

True

In [51]:
2003 in frame3.index

False

![](./index的方法和属性.png)

## 重新索引
`pandas`对象的一个重要方法是`reindex`，其作用是创建一个适应新索引的新对象。

In [52]:
obj = Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

调用该`Series`的`reindex`将会根据新索引进行重排。若某个索引值当前不存在，就引入缺失值：

In [53]:
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

In [54]:
obj.reindex(['a', 'b', 'c', 'd', 'e'], fill_value=0)   # 让缺失值为0

a   -5.3
b    7.2
c    3.6
d    4.5
e    0.0
dtype: float64

对于时间序列这样的有序数据，重新索引时可能需要做一些插值处理。`method`选项即可达到此目的，如，使用`ffill`可以实现前向值填充：

In [55]:
obj3 = Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])
obj3.reindex(range(6), method='ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

![](./reindex的(插值)method选项.png)

对于`DataFrame`，`reindex`可以修改（行）索引、列，或两个都修改。若仅传入一个序列，则会重新索引行：

In [56]:
frame = DataFrame(np.arange(9).reshape((3, 3)), index=['a', 'c', 'd'], columns=['Ohio', 'Texas', 'California'])
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [57]:
frame2 = frame.reindex(['a', 'b', 'c', 'd'])
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


使用`columns`关键字即可重新索引列：

In [58]:
states = ['Texas', 'Utah', 'California']
frame.reindex(columns=states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


也可同时对行和列进行重新索引，而插值则只能按行应用（即轴0）：

In [59]:
frame.reindex(index=['a', 'b', 'c', 'd'], columns=states)

,Texas,Utah,California
a,1.0,NaN,2.0
b,NaN,NaN,NaN
c,4.0,NaN,5.0
d,7.0,NaN,8.0


利用`loc`的标签索引功能，重新索引任务可以变得更简洁：

In [60]:
frame.loc['a']

Ohio          0
Texas         1
California    2
Name: a, dtype: int32

![](./reindex函数的参数.png)

## 丢弃指定轴上的项
`drop`方法返回一个在指定轴上删除了指定值的新对象：

In [61]:
obj = Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])
new_obj = obj.drop('c')
new_obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [62]:
obj.drop(['d', 'c'])

a    0.0
b    1.0
e    4.0
dtype: float64

对于`DataFrame`，可以删除任意轴上的索引值：

In [63]:
data = DataFrame(np.arange(16).reshape((4, 4)), 
                 index=['Ohio', 'Colorado', 'Utah', 'New York'],
                columns=['one', 'two', 'three', 'four'])
data.drop(['Colorado', 'Ohio'])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


In [64]:
data.drop('two', axis=1)  # 轴为1,按列计算，轴为0按行计算

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


In [65]:
data.drop(['two', 'four'], axis=1)

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


## 索引、选取和过滤
`Series`索引（obj[...]）的工作方式类似于`Numpy`数组的索引，只不过`Series`的索引值不只是整数：

In [66]:
obj = Series(np.arange(4.), index=['a', 'b', 'c', 'd'])
obj['b']

1.0

In [67]:
obj[1]

1.0

In [68]:
obj[2:4]

c    2.0
d    3.0
dtype: float64

In [69]:
obj[['b', 'a', 'd']]

b    1.0
a    0.0
d    3.0
dtype: float64

In [70]:
obj[[1,3]]

b    1.0
d    3.0
dtype: float64

In [71]:
obj[obj < 2]

a    0.0
b    1.0
dtype: float64

利用标签的切片运算与普通的`Python`切片运算不同，其末端是包含的(即闭区间)：

In [72]:
obj['b':'c']

b    1.0
c    2.0
dtype: float64

设置方式：

In [73]:
obj['b':'c'] = 5
obj

a    0.0
b    5.0
c    5.0
d    3.0
dtype: float64

对`DataFrame`进行索引其实就是获取一个或多个`列`：

In [74]:
data = DataFrame(np.arange(16).reshape(4, 4),
                index=['Ohio', 'Colorado', 'Utah', 'New York'],
                columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [75]:
data['two']

Ohio         1
Colorado     5
Utah         9
New York    13
Name: two, dtype: int32

In [76]:
data[['three', 'one']]

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
New York,14,12


这种索引方式有几个特殊的情况。首先通过切片或布尔型数组选取`行`：

In [77]:
data[:2]

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


In [78]:
data[data['three'] > 5]

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


另一种用法是通过布尔型`DataFrame`（如下面这个由标量进行比较运算得出的）进行索引：

In [79]:
data < 5

,one,two,three,four
Ohio,True,True,True,True
Colorado,True,False,False,False
Utah,False,False,False,False
New York,False,False,False,False


In [80]:
data[data < 5] = 0
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


这段代码的目的是使`DataFrame`在语法上更像`ndarray`。为了在`DataFrame`的行上进行标签索引，引入了专门的索引字段`loc`和`iloc`。它使你可通过`Numpy`式的标记法以及轴标签从`DataFrame`中选取行和列的子集。`iloc`为数字，`loc`为轴标签

In [81]:
data.loc['Colorado', ['two', 'three']]

two      5
three    6
Name: Colorado, dtype: int32

In [82]:
data.iloc[2, [3, 0, 1]]

four    11
one      8
two      9
Name: Utah, dtype: int32

In [83]:
data.iloc[2]

one       8
two       9
three    10
four     11
Name: Utah, dtype: int32

In [84]:
data.iloc[[1, 2], [3, 0, 1]]

,four,one,two
Colorado,7,0,5
Utah,11,8,9


In [85]:
data.loc[['Utah', 'Colorado'],[ 'one', 'two']]

,one,two
Utah,8,9
Colorado,0,5


In [86]:
data.loc[:'Utah', 'two']

Ohio        0
Colorado    5
Utah        9
Name: two, dtype: int32

In [87]:
data.iloc[:, :3][data.three > 5]  # [行， 列][条件]

,one,two,three
Colorado,0,5,6
Utah,8,9,10
New York,12,13,14


In [88]:
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


![](./DataFrame的索引选项.png)
(图中`ix`应相应替换为`loc`或`iloc`，如上)  
`get_value`方法是选取，`set_value`方法是设置

## 算术运算和数据对齐
`pandas`最重要的一个功能是，他可以对不同索引的对象进行算术运算。在将对象相加时，若存在不同的索引对，则结果的索引就是该索引对的并集：

In [89]:
s1 = Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])
s2 = Series([-2.1, 3.6, -1.5, 4, 3.1], index=['a', 'c', 'e', 'f', 'g'])
s1

a    7.3
c   -2.5
d    3.4
e    1.5
dtype: float64

In [90]:
s2

a   -2.1
c    3.6
e   -1.5
f    4.0
g    3.1
dtype: float64

In [91]:
s1 + s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

自动的数据对齐操作在不重叠的索引处引入了`NA`值。缺失值会在算术运算过程中传播。  
对于`DataFrame`，对齐操作会同时发生在行和列上：

In [92]:
df1 = DataFrame(np.arange(9.).reshape((3, 3)),
               columns=list('bcd'),
               index=['Ohio', 'Texas', 'Colorado'])
df2 = DataFrame(np.arange(12.).reshape((4,3)),
               columns=list('bde'),
               index=['Utah', 'Ohio', 'Texas', 'Oregon'])
df1

,b,c,d
Ohio,0.0,1.0,2.0
Texas,3.0,4.0,5.0
Colorado,6.0,7.0,8.0


In [93]:
df2

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


把他们相加后返回一个新的`DataFrame`，其索引和列为原来那两个`DataFrame`的并集（且相加）：

In [94]:
df1 + df2

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN


## 在算术方法中填充值
在对不同索引的对象进行算术运算时，当一个对象中某个轴标签在另一个对象中找不到时填充一个特殊值（如0）：

In [95]:
df1 = DataFrame(np.arange(12.).reshape((3, 4)), columns=list('abcd'))
df2 = DataFrame(np.arange(20.).reshape((4, 5)), columns=list('abcde'))
df1

,a,b,c,d
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0


In [96]:
df2

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.0,8.0,9.0
2,10.0,11.0,12.0,13.0,14.0
3,15.0,16.0,17.0,18.0,19.0


将它们相加时，没有重叠的位置就会产生`NA`值：

In [97]:
df1 + df2

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,11.0,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


使用`df1`的`add`方法，传入`df2`以及一个`fill_value`参数：

In [98]:
df1.add(df2, fill_value=0)  # 将NA值处填充为原值，并使该原值+0(可设置)

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,11.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


与此类似，在对`Series`或`DataFrame`重新索引时，也可指定一个填充值：

In [99]:
df1.reindex(columns=df2.columns, fill_value=0)

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,0
1,4.0,5.0,6.0,7.0,0
2,8.0,9.0,10.0,11.0,0


![](./灵活的算术方法.png)

## DataFrame和Series之间的运算
计算一个二维数组与其某行之间的差：

In [100]:
arr = np.arange(12.).reshape((3, 4))
arr

array([[ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11.]])

In [101]:
arr[0]

array([0., 1., 2., 3.])

In [102]:
arr - arr[0]

array([[0., 0., 0., 0.],
       [4., 4., 4., 4.],
       [8., 8., 8., 8.]])

这就叫做`广播`。  
`DataFrame`和`Series`之间的运算差不多也是如此：

In [103]:
frame = DataFrame(np.arange(12.).reshape((4, 3)),
                 columns=list('bde'),
                 index=['Utah', 'Ohio', 'Texas', 'Oregon'])
series = frame.iloc[0]
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [104]:
series

b    0.0
d    1.0
e    2.0
Name: Utah, dtype: float64

默认情况下，`DataFrame`和`Series`之间的算术运算会将`Series`的索引匹配到`DataFrame`的列，然后沿着行一直向下广播：

In [105]:
frame - series

,b,d,e
Utah,0.0,0.0,0.0
Ohio,3.0,3.0,3.0
Texas,6.0,6.0,6.0
Oregon,9.0,9.0,9.0


若某个索引值在`DataFrame`的列或`Series`的索引中找不到，则参与运算的两个对象会被重新索引以形成并集：

In [106]:
series2 = Series(range(3), index=['b', 'e', 'f'])
frame + series2

,b,d,e,f
Utah,0.0,NaN,3.0,NaN
Ohio,3.0,NaN,6.0,NaN
Texas,6.0,NaN,9.0,NaN
Oregon,9.0,NaN,12.0,NaN


若希望匹配行且在行上广播，则必须使用算术运算方法：

In [107]:
series3 = frame['d']
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [108]:
series3

Utah       1.0
Ohio       4.0
Texas      7.0
Oregon    10.0
Name: d, dtype: float64

In [109]:
frame.sub(series3, axis=0)

,b,d,e
Utah,-1.0,0.0,1.0
Ohio,-1.0,0.0,1.0
Texas,-1.0,0.0,1.0
Oregon,-1.0,0.0,1.0


传入的轴号就是希望匹配的轴。此例中，目的是匹配`DataFrame`的行索引并进行广播。

## 函数应用的映射
`Numpy`的`ufuncs`(元素级数组方法)也可用于操作`pandas`对象：

In [110]:
frame = DataFrame(np.random.randn(4, 3),
                 columns=list('bde'),
                 index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,-0.636866,1.908984,-0.831774
Ohio,1.892730,-1.629744,-0.844510
Texas,0.711505,-0.971947,0.040689
Oregon,-0.803299,-1.260870,1.608431


In [111]:
np.abs(frame)

,b,d,e
Utah,0.636866,1.908984,0.831774
Ohio,1.892730,1.629744,0.844510
Texas,0.711505,0.971947,0.040689
Oregon,0.803299,1.260870,1.608431


另一个常见的操作是，将函数应用到由各列或行所形成的一维数组上。`DataFrame`的`apply`方法即可实现此功能：

In [112]:
f = lambda x: x.max() - x.min()
frame.apply(f)

b    2.696028
d    3.538728
e    2.452941
dtype: float64

In [113]:
frame.apply(f, axis=1)

Utah      2.740758
Ohio      3.522474
Texas     1.683452
Oregon    2.869301
dtype: float64

许多最为常见的数组统计功能都被实现成`DataFrame`的方法(如`sum`和`mean`)，因此无需使用apply方法。  
除标量值外，传递给`apply`的函数还可以返回由多个值组成的`Series`:

In [114]:
def f(x):
    return Series([x.min(), x.max()], index=['min', 'max'])
frame.apply(f)

,b,d,e
min,-0.803299,-1.629744,-0.844510
max,1.892730,1.908984,1.608431


In [115]:
frame

,b,d,e
Utah,-0.636866,1.908984,-0.831774
Ohio,1.892730,-1.629744,-0.844510
Texas,0.711505,-0.971947,0.040689
Oregon,-0.803299,-1.260870,1.608431


此外，元素级的`Python`函数也是可用的。如想得到`frame`中各个浮点值的格式化字符串，使用`applymap`即可：

In [116]:
format = lambda x: '%.2f' % x
frame.applymap(format)

,b,d,e
Utah,-0.64,1.91,-0.83
Ohio,1.89,-1.63,-0.84
Texas,0.71,-0.97,0.04
Oregon,-0.80,-1.26,1.61


之所以叫`applymap`，是因为`Series`有一个用于应用元素级函数的`map`方法：

In [117]:
frame['e'].map(format)

Utah      -0.83
Ohio      -0.84
Texas      0.04
Oregon     1.61
Name: e, dtype: object

## 排序和排名
根据条件对数据集排序也是一种重要的内置运算。要对行或列索引进行排序(按字典顺序)，可用`sort_index`方法，他将返回一个已排序的新对象：

In [118]:
obj = Series(range(4), index=['d', 'a', 'b', 'c'])
obj.sort_index()

a    1
b    2
c    3
d    0
dtype: int64

对`DataFrame`，则可以根据任意一个轴上的索引进行排序：

In [119]:
frame = DataFrame(np.arange(8).reshape((2, 4)),
                 index=['three', 'one'],
                 columns=['d', 'a', 'b', 'c'])
frame.sort_index()

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


In [120]:
frame.sort_index(axis=1)

,a,b,c,d
three,1,2,3,0
one,5,6,7,4


数据默认是按升序排序的，但也可以降序排序：

In [121]:
frame.sort_index(axis=1, ascending=False)

,d,c,b,a
three,0,3,2,1
one,4,7,6,5


若要按值对`Series`进行排序，可使用其`sort_values`方法：

In [122]:
obj = Series([4, 7, -3, 2])
obj.sort_values()

2   -3
3    2
0    4
1    7
dtype: int64

在排序时，任何缺失值默认都会被放到`Series`的末尾：

In [123]:
obj = Series([4, np.nan, 7, np.nan, -3, 2])
obj.sort_values()

4   -3.0
5    2.0
0    4.0
2    7.0
1    NaN
3    NaN
dtype: float64

在`DataFrame`上，希望根据一个或多个列中的值进行排序。将一个或多个列的名字传递给`by`选项即可达到该目的：

In [124]:
frame = DataFrame({'b': [4, 7, -3, 2], 'a': [0, 1, 0, 1]})
frame

,b,a
0,4,0
1,7,1
2,-3,0
3,2,1


In [125]:
frame.sort_values(by='b')

,b,a
2,-3,0
3,2,1
0,4,0
1,7,1


要根据多个列进行排序，传入名称的列表即可：

In [126]:
frame.sort_values(by=['a', 'b'])

,b,a
2,-3,0
0,4,0
3,2,1
1,7,1


排名跟排序关系密切，且它会增设一个排名值(从1开始，一直到数组中有效数据的数量)。他跟`numpy.argsort`产生的间接排序索引差不多，只不过它可根据某种规则破坏平级关系。接下来介绍`Series`和`DataFrame`的`rank`方法。默认下，`rank`是通过"为各组分配一个平均排名"的方式破坏平级关系的：

In [127]:
obj = Series([7, -5, 7, 4, 2, 0, 4])  # 7,7为6,7（(6+7)/2=6.5）,4,4为4,5((4+5)/2=4.5)
obj.rank()

0    6.5
1    1.0
2    6.5
3    4.5
4    3.0
5    2.0
6    4.5
dtype: float64

也可以根据值在原数据中出现的顺序给出排名(类似于稳定排序)：

In [128]:
obj.rank(method='first')

0    6.0
1    1.0
2    7.0
3    4.0
4    3.0
5    2.0
6    5.0
dtype: float64

也可以按降序排名：

In [129]:
obj.rank(ascending=False, method='max')  # ascending(升序)=False

0    2.0
1    7.0
2    2.0
3    4.0
4    5.0
5    6.0
6    4.0
dtype: float64

`DataFrame`可在行或列上计算排名：

In [130]:
frame = DataFrame({'b': [4.3, 7, -3, 2],
                  'a': [0, 1, 0, 1],
                  'c': [-2, 5, 8, -2.5]})
frame

,b,a,c
0,4.3,0,-2.0
1,7.0,1,5.0
2,-3.0,0,8.0
3,2.0,1,-2.5


In [131]:
frame.rank(axis=0)

,b,a,c
0,3.0,1.5,2.0
1,4.0,3.5,3.0
2,1.0,1.5,4.0
3,2.0,3.5,1.0


In [132]:
frame.rank(axis=1)

,b,a,c
0,3.0,2.0,1.0
1,3.0,1.0,2.0
2,1.0,2.0,3.0
3,3.0,2.0,1.0


![](./排名时用于破坏平级关系的method选项.png)

## 带有重复值的轴索引
许多`pandas`函数(如`reindex`)都要求标签唯一，但并非强制。下面这个带有重复索引值的`Series`：

In [133]:
obj = Series(range(5), index=['a', 'a', 'b', 'b', 'c'])
obj

a    0
a    1
b    2
b    3
c    4
dtype: int64

索引的`is_unique`属性可以告诉你它的值是否是唯一的：

In [134]:
obj.index.is_unique

False

若某个索引对应多个值，则返回一个`Series`；而对应单个值的，则返回一个标量值：

In [135]:
obj['a']

a    0
a    1
dtype: int64

In [136]:
obj['c']

4

对`DataFrame`的行进行索引时也是如此：

In [137]:
df = DataFrame(np.random.randn(4, 3), index=['a', 'a', 'b', 'b'])
df

,0,1,2
a,0.281329,-1.309192,0.506369
a,0.272204,0.648514,0.130661
b,0.796734,-0.338518,0.202949
b,0.107937,-0.005692,0.855338


In [138]:
df.loc['b']

,0,1,2
b,0.796734,-0.338518,0.202949
b,0.107937,-0.005692,0.855338


## 汇总和计算描述统计
`pandas`有一组常用的数学和统计方法，它们大部分都属于约简和汇总统计，用于从`Series`中提取单个值(如`sum`或`mean`)或从`DataFrame`的行或列中提取一个`Series`。与对应的`Numpy`数组方法相比，它们都是基于没有缺失数据的假设而构建的：

In [139]:
df = DataFrame([[1.4, np.nan], [7.1, -4.5],
               [np.nan, np.nan], [0.75, -1.3]],
              index=['a', 'b', 'c', 'd'],
              columns=['one', 'two'])
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


调用`DataFrame`的`sum`方法将会返回一个含有列小计(列之和)的`Series`:

In [140]:
df.sum()

one    9.25
two   -5.80
dtype: float64

传入axis=1将会按行进行求和运算：

In [141]:
df.sum(axis=1)

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

`NA`值会自动被排除，除非整个切片(这里指的是行或列)都是`NA`。通过`skipna`选项可以禁用该功能：

In [142]:
df.mean(axis=1, skipna=False)  # 平均值

a      NaN
b    1.300
c      NaN
d   -0.275
dtype: float64

![](./简约方法的选项.png)

有些方法(如`idxmin`和`idxmax`)返回的是间接统计(比如达到最小值或最大值的索引):

In [143]:
df.idxmax()

one    b
two    d
dtype: object

另一些方法则是累计型的：

In [144]:
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


In [145]:
df.cumsum()  # 从上到下累加

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


还有一种方法，它既不是约简型也不是累计型。`describe`就是一个例子,它用于一次性产生多个汇总统计 ：

In [146]:
df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


对于非数值型数据,`describe`会产生另外一种汇总统计：

In [147]:
obj = Series(['a', 'a', 'b', 'c'] * 4)
obj.describe()

count     16
unique     3
top        a
freq       8
dtype: object

![](./描述和汇总统计.png)

## 相关系数和协方差
有些汇总统计（如相关系数和协方差）是通过参数对计算出来的。我们来看几个`DataFrame` ,它们的数据来自`Yahoo ! Fina nce`的股票价格和成交猫：  
　　1）python中有一种存储方式，可以存储为.pkl文件。

　　2）该存储方式，可以将python项目过程中用到的一些暂时变量、或者需要提取、暂存的字符串、列表、字典等数据保存起来。

　　3）保存方式就是保存到创建的.pkl文件里面。

　　4）然后需要使用的时候再 open，load。

In [279]:
price = pd.read_pickle('./yahoo_price.pkl')  # 使用pandas库的pd.read_pickle读取pickle数据
volume = pd.read_pickle('./yahoo_volume.pkl')

In [280]:
import pandas_datareader.data as web
all_data = {ticker: web.get_data_yahoo(ticker)
            for ticker in ['AAPL', 'IBM', 'MSFT', 'GOOG']}

price = pd.DataFrame({ticker: data['Adj Close']
                     for ticker, data in all_data.items()})
volume = pd.DataFrame({ticker: data['Volume']
                      for ticker, data in all_data.items()})

接下来计算价格的百分比数变化：

In [150]:
price

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-04,27.990226,313.062468,113.304536,25.884104
2010-01-05,28.038618,311.683844,111.935822,25.892466
2010-01-06,27.592626,303.826685,111.208683,25.733566
2010-01-07,27.541619,296.753749,110.823732,25.465944
2010-01-08,27.724725,300.709808,111.935822,25.641571
...,...,...,...,...
2016-10-17,117.550003,779.960022,154.770004,57.220001
2016-10-18,117.470001,795.260010,150.720001,57.660000
2016-10-19,117.120003,801.500000,151.259995,57.529999


In [151]:
volume

,AAPL,GOOG,IBM,MSFT
Date,,,,
2010-01-04,123432400,3927000,6155300,38409100
2010-01-05,150476200,6031900,6841400,49749600
2010-01-06,138040000,7987100,5605300,58182400
2010-01-07,119282800,12876600,5840600,50559700
2010-01-08,111902700,9483900,4197200,51197400
...,...,...,...,...
2016-10-17,23624900,1089500,5890400,23830000
2016-10-18,24553500,1995600,12770600,19149500
2016-10-19,20034600,116600,4632900,22878400


In [152]:
returns = price.pct_change()   # 计算百分比变化
returns.tail()  # tail()方法一般用来对数据集进行矩阵形式的显示，默认显示为数据集的最后5行

,AAPL,GOOG,IBM,MSFT
Date,,,,
2016-10-17,-0.000680,0.001837,0.002072,-0.003483
2016-10-18,-0.000681,0.019616,-0.026168,0.007690
2016-10-19,-0.002979,0.007846,0.003583,-0.002255
2016-10-20,-0.000512,-0.005652,0.001719,-0.004867
2016-10-21,-0.003930,0.003011,-0.012474,0.042096


`Series`的`corr`方法用于计算两个`Series`中重叠的、非`NA`的、按索引对齐的值的相关系数。与此类似，`cov`用于计算协方差：

In [153]:
returns.MSFT.corr(returns.IBM)

0.49976361144151155

In [154]:
returns.MSFT.cov(returns.IBM)

8.870655479703549e-05

`DataFrame`的`corr`和`COV`方法将以`DataFrame`的形式返回完整的相关系数或协方差矩阵：

In [155]:
returns.corr()

,AAPL,GOOG,IBM,MSFT
AAPL,1.000000,0.407919,0.386817,0.389695
GOOG,0.407919,1.000000,0.405099,0.465919
IBM,0.386817,0.405099,1.000000,0.499764
MSFT,0.389695,0.465919,0.499764,1.000000


In [156]:
returns.cov()

,AAPL,GOOG,IBM,MSFT
AAPL,0.000277,0.000107,0.000078,0.000095
GOOG,0.000107,0.000251,0.000078,0.000108
IBM,0.000078,0.000078,0.000146,0.000089
MSFT,0.000095,0.000108,0.000089,0.000215


利用`DataFrame`的`corrwith`方法，你可以计算其列或行跟另一个`Series`或`DataFrame`之间的相关系数。传入一个`Series`将会返回一个相关系数值Series （针对各列进行计算）：

In [157]:
returns.corrwith(returns.IBM)

AAPL    0.386817
GOOG    0.405099
IBM     1.000000
MSFT    0.499764
dtype: float64

传入一个`DataFrame`则会计算按列名配对的相关系数。这里，计算百分比变化与成交量的相关系数：

In [158]:
returns.corrwith(volume)

AAPL   -0.075565
GOOG   -0.007067
IBM    -0.204849
MSFT   -0.092950
dtype: float64

传入`axis = l `即可按行进行计算。无论如何，在计 算相关系数之前，所有的数据项都会桉标签对齐。

## 唯一值、值计数以及成员资格
还有一类方法可从一维`Series`的值中抽取信息。以下面这个`Series`为例：

In [159]:
obj = Series(['c', 'a', 'd', 'a', 'a', 'b', 'b', 'c', 'c'])

第一个函数是unique，它可得到`Series`中的唯一值数组：

In [160]:
uniques = obj.unique()
uniques

array(['c', 'a', 'd', 'b'], dtype=object)

返回的唯一值是未排序的，若需要的话，可对结果再次进行排序`(uniques.sort())`。`value_counts`用于计算一个`Series`中各值出现的频率：

In [161]:
obj.value_counts()

c    3
a    3
b    2
d    1
dtype: int64

为了便于查看，结果`Series`是按值频率降序排列的。`value_counts`还是一个顶级`pandas`方法，可用于任何数组或序列：

In [162]:
pd.value_counts(obj.values, sort=False)

a    3
d    1
c    3
b    2
dtype: int64

最后是`isin`，它用于判断矢量化集合的成员资格，可用于选取`Series`中或`DataFrame`列中数据的子集：

In [163]:
mask = obj.isin(['b', 'c'])  # isin()接受一个列表，判断该列中元素是否在列表中
mask

0     True
1    False
2    False
3    False
4    False
5     True
6     True
7     True
8     True
dtype: bool

In [164]:
obj[mask]

0    c
5    b
6    b
7    c
8    c
dtype: object

![](./唯一值、值计数、成员资格方法.png)

若希望得到`DataFrame`中多个相关列的一张柱状图：

In [165]:
data = DataFrame({'Qu1': [1, 3, 4, 3, 4],
                 'Qu2': [2, 3, 1, 2, 3],
                 'Qu3': [1, 5, 2, 4, 4]})
data

,Qu1,Qu2,Qu3
0,1,2,1
1,3,3,5
2,4,1,2
3,3,2,4
4,4,3,4


将`pandas.value_counts`传给该`DataFrame`的`apply`函数，就会出现：  
`pandas` 的 `apply() `函数可以作用于 `Series `或者整个 `DataFrame`，功能也是自动遍历整个 `Series` 或者 `DataFrame`, 对每一个元素运行指定的函数。  
`value_counts()`是一种查看表格某列中有多少个不同值的快捷方法，并计算每个不同值有在该列中有多少重复值。

In [166]:
result = data.apply(pd.value_counts).fillna(0)
result  # 如Qu1中1出现一次，2没有出现，3出现两次，4出现两次，5没有出现，fillna(0)将所有na值置为0

,Qu1,Qu2,Qu3
1,1.0,1.0,1.0
2,0.0,2.0,1.0
3,2.0,2.0,0.0
4,2.0,0.0,2.0
5,0.0,0.0,1.0


## 处理缺失数据
`pandas`对象上的所有描述统计都排除了缺失数据，正如我们在本章稍早的地方所看到的那样。  
`pandas`使用浮点值`NaN( Not a Number )` 表示浮点和非浮点数组中的缺失数据。它只是一个便于袚检测出来的标记而已：

In [167]:
string_data = Series(['arrdvark', 'artichoke', np.nan, 'avocado'])
string_data

0     arrdvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [168]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

`Python`内置的`None`值也会被当作`NA`处理：

In [169]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

`pandas`的`NA`表现形式很简单也很可靠。由于`NumPy`的数据类型体系中缺乏真正的`NA`数据类型或位模式，所以它是我能想到的 最佳解决方案 （一套简单的`API`以及足够全面的性能特征）。随着`NumPy`的不断发展，这个问题今后可能会发生变化。  
![](./NA处理方法.png)

## 滤除缺失数据
对于一个`Series`，`dropna`返回一个仅含非空数据和索引值的`Series`:

In [170]:
from numpy import nan as NA
data = Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

也可通过布尔型索引达到这个目的：

In [171]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

对于`DataFrame`，`dropna`默认丢弃任何含有缺失值的行：

In [172]:
data = DataFrame([[1., 6.5, 3.], [1., NA, NA], 
                  [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [173]:
cleaned

,0,1,2
0,1.0,6.5,3.0


传入`how='all'`将只丢弃全为`NA`的那些行：

In [174]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


用这种方式丢弃列，只需传入`axis=1`即可：

In [175]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [176]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


另一个滤除`DataFrame`行的问题涉及时间序列数据。假设你只想留下一部分观测数据，可以用`thresh`参数实现此目的：

In [177]:
df = DataFrame(np.random.randn(7, 3))
df.loc[:4, 1] = NA; df.loc[:2, 2] = NA
df

,0,1,2
0,1.971179,NaN,NaN
1,0.695620,NaN,NaN
2,-0.762664,NaN,NaN
3,0.629364,NaN,-0.067788
4,-2.364725,NaN,-0.028939
5,-0.059942,-1.189631,-0.056230
6,-0.225645,2.130922,-0.637147


In [178]:
df.dropna(thresh=2)

,0,1,2
3,0.629364,NaN,-0.067788
4,-2.364725,NaN,-0.028939
5,-0.059942,-1.189631,-0.056230
6,-0.225645,2.130922,-0.637147


## 填充缺失数据
通过一个常数调用`fillna`就会将缺失值替换为那个常数值：

In [179]:
df.fillna(0)

,0,1,2
0,1.971179,0.000000,0.000000
1,0.695620,0.000000,0.000000
2,-0.762664,0.000000,0.000000
3,0.629364,0.000000,-0.067788
4,-2.364725,0.000000,-0.028939
5,-0.059942,-1.189631,-0.056230
6,-0.225645,2.130922,-0.637147


若通过一个字典调用`fillna`，就可以实现对不同的列填充不同的值：

In [180]:
df.fillna({1: 0.5, 3: -1})

,0,1,2
0,1.971179,0.500000,NaN
1,0.695620,0.500000,NaN
2,-0.762664,0.500000,NaN
3,0.629364,0.500000,-0.067788
4,-2.364725,0.500000,-0.028939
5,-0.059942,-1.189631,-0.056230
6,-0.225645,2.130922,-0.637147


`fillna`默认会返回新对象，但也可对现有对象进行就地修改：

In [181]:
# 总是返回被填充对象的引用
_ = df.fillna(0, inplace=True)
df

,0,1,2
0,1.971179,0.000000,0.000000
1,0.695620,0.000000,0.000000
2,-0.762664,0.000000,0.000000
3,0.629364,0.000000,-0.067788
4,-2.364725,0.000000,-0.028939
5,-0.059942,-1.189631,-0.056230
6,-0.225645,2.130922,-0.637147


对`reindex`有效的那些插值方法也可用于`fillna`:

In [182]:
df = DataFrame(np.random.randn(6, 3))
df.loc[2:, 1] = NA; df.loc[4:, 2] = NA
df

,0,1,2
0,-2.142107,-1.071421,0.922186
1,-0.203760,0.419107,0.259369
2,1.790814,NaN,0.840225
3,1.315848,NaN,0.222644
4,1.411842,NaN,NaN
5,-0.310092,NaN,NaN


In [183]:
df.fillna(method='ffill')

,0,1,2
0,-2.142107,-1.071421,0.922186
1,-0.203760,0.419107,0.259369
2,1.790814,0.419107,0.840225
3,1.315848,0.419107,0.222644
4,1.411842,0.419107,0.222644
5,-0.310092,0.419107,0.222644


In [184]:
df.fillna(method='ffill', limit=2)  # limit:范围，此处是向下填充两行

,0,1,2
0,-2.142107,-1.071421,0.922186
1,-0.203760,0.419107,0.259369
2,1.790814,0.419107,0.840225
3,1.315848,0.419107,0.222644
4,1.411842,NaN,0.222644
5,-0.310092,NaN,0.222644


可利用`fillna`实现许多别的功能。如，可传入`Series`的平均值或中位数：

In [185]:
data = Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

![](./fillna函数的参数.png)

## 层次化索引
层次化索引使你能在一个轴上拥有多个（两个以上）索引级别。抽象点说，它使你能以低维度形式处理高维度数据。我们先来看一个简单的例子：  创建一个`Series`,并用一个由列表或数组组成的列表作为索引。

In [186]:
data = Series(np.random.randn(10),
              index=[['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'd', 'd'],
                     [1, 2, 3, 1, 2, 3, 1, 2, 2, 3]])
data

a  1   -0.729006
   2    1.863161
   3   -0.889452
b  1   -0.272238
   2   -0.412815
   3   -1.244291
c  1   -0.196342
   2    0.042045
d  2   -1.608111
   3    0.583306
dtype: float64

这就是带有`MultiIndex`索引的`Series`的格式化输出形式。索引之间的“间隔”表示“直接使用上面的标签”:

In [187]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 2),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

对于一个层次化索引的对象，选取数据子集的操作很简单：

In [188]:
data['b']

1   -0.272238
2   -0.412815
3   -1.244291
dtype: float64

In [189]:
data['b':'c']

b  1   -0.272238
   2   -0.412815
   3   -1.244291
c  1   -0.196342
   2    0.042045
dtype: float64

In [190]:
data.loc[['b', 'd']]

b  1   -0.272238
   2   -0.412815
   3   -1.244291
d  2   -1.608111
   3    0.583306
dtype: float64

甚至还可在“内层”中选取：

In [191]:
data[:, 2]

a    1.863161
b   -0.412815
c    0.042045
d   -1.608111
dtype: float64

层次化索引在数据重塑和基于分组的操作（如透视表生成）中扮演着重要的角色。比如说，这段数据可以通过其`unstack`方法披重新安排到一个`DataFrame`中：

In [192]:
data.unstack()

,1,2,3
a,-0.729006,1.863161,-0.889452
b,-0.272238,-0.412815,-1.244291
c,-0.196342,0.042045,NaN
d,NaN,-1.608111,0.583306


`unstack`的逆运算是`stack`:

In [193]:
data.unstack().stack()

a  1   -0.729006
   2    1.863161
   3   -0.889452
b  1   -0.272238
   2   -0.412815
   3   -1.244291
c  1   -0.196342
   2    0.042045
d  2   -1.608111
   3    0.583306
dtype: float64

对于一个`DataFrame`，每条轴都可以有分层索引：

In [194]:
frame = DataFrame(np.arange(12).reshape((4, 3)),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

各层都可以有名字（可以是字符串，也可以是别的`Python`对象）。如果指定了名称， 它们就会显示在控制台输出中（不要将索引名称跟轴标签混为一谈！）:

In [195]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

由于有了分部的列索引，因此可以轻松选取列分组：

In [196]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

可单独创建`MultiIndex`然后复用。上面那个`DataFrame`中的(分级的)列可以这样创建：

In [197]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                       names=['state', 'color'])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

## 重排分级顺序
有时，你需要重新调整某条轴上各级别的顺序，或根据指定级别上的值对数据进行排序。`swaplevel`接受两个级别编号或名称，并返回一个互换了级别的新对象（但数据不会发生变化）：

In [198]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

`sort_index()`函数对索引进行排序，例如：  
按`frame`的行索引进行排序

In [212]:
frame.sort_index(level=0, ascending=True)  # 默认为行索引，行索引第0列排序

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [213]:
frame.sort_index(level=1, ascending=True)  # 行索引第1列排序

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

按`frame`的列索引进行排序

In [217]:
frame.sort_index(level=0, axis=1, ascending=False)  # 列索引的第零行按首字母降序排序

state     Ohio       Colorado
color      Red Green    Green
key1 key2                    
a    1       1     0        2
     2       4     3        5
b    1       7     6        8
     2      10     9       11

In [216]:
frame.sort_index(level=1, axis=1, ascending=False)  # 列索引的第一行按首字母降序排序

state     Ohio       Colorado
color      Red Green    Green
key1 key2                    
a    1       1     0        2
     2       4     3        5
b    1       7     6        8
     2      10     9       11

In [218]:
frame.swaplevel('key1', 'key2').sort_index(level=0, ascending=True)  # 交换key1,key2，，行索引第0列排序

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

`sort_values()`函数对值进行排序

In [235]:
frame.sort_values(by=('Ohio','Red'), ascending=False)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
b    2        9  10       11
     1        6   7        8
a    2        3   4        5
     1        0   1        2

In [240]:
frame.sort_values(by=('a',1), ascending=False, axis=1)

state     Colorado Ohio      
color        Green  Red Green
key1 key2                    
a    1           2    1     0
     2           5    4     3
b    1           8    7     6
     2          11   10     9

In [241]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

## 根据级别汇总统计
许多对`DataFrame`和`Series`的描述和汇总统计都有一个`level`选项，它用于指定在某条轴上求和的级别。再以上面那个`DataFrame`为例，我们可以根据行或列上的级别来进行求和，如下所示 ：

In [239]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [242]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

这其实是利用了`pandas`的`groupby`功能。

## 使用DataFrame的列
想要将`DataFrame`的一个或多个列当做行索引来用，或者可能希望将行索引变成`DataFrame`的列。以下面这个`DataFrame`为例：

In [244]:
frame = DataFrame({'a': range(7), 'b': range(7, 0, -1),
                   'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                   'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


`DataFrame`的`set_index`函数会将其一个或多个列转换为行索引,并创建一个新的
`DataFrame`:

In [245]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

默认情况下，那些列会从`DataFrame`中移除，但也可以将其保留下来：

In [246]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

`reset_index`功能跟`set_index`刚好相反，层次化索引的级别会被转移到列里面：

In [247]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 整数索引
操作由整数索引的`pandas`对象常常会让新手抓狂，因为它们跟内置的`Python`数据结构
（如列表和元组）在索引语义上有些不同。例如，你可能认为下面这段代码不会产生一个错误：

```ser = Series(np.arange(3.))
ser[-1]```

在这种情况下，虽然`pandas`会“求助于”整数索引， 但没有哪种方法（至少我就不知道）能够既不引入任何`bug`又安全有效地解决该问题。这里，我们有一个含有0、1、2的索引，但是很难推断出用户想要什么（基千标签或位置的索引） ：

In [250]:
ser = Series(np.arange(3.))
ser

0    0.0
1    1.0
2    2.0
dtype: float64

相反，对于一个非整数索引，就没有这样的歧义：

In [251]:
ser2 = Series(np.arange(3.), index=['a', 'b', 'c'])
ser2[-1]

2.0

为了保持良好的一致性，如果你的轴索引含有索引器，那么根据整数进行数据选取的操 作将总是面向标签的。这也包括用`loc`进行切片：

In [252]:
ser.loc[:1]

0    0.0
1    1.0
dtype: float64

如果你需要可靠的、不考虑索引类型的、基于位置的索引，可以使用`Series`的`iat`方法和`DataFrame`的`iloc`方法：

In [267]:
ser3 = Series(range(3), index=[-5, 1, 3])
ser3.iat[2]

2

In [272]:
frame = DataFrame(np.arange(6).reshape(3, 2), index=[2, 0, 1])
frame.iloc[1]

0    2
1    3
Name: 0, dtype: int32